# Download Advanced Search matches - A1000
This notebook offers the option of downloading multiple files matched by the Advanced Search API.

### Used A1000 functions
- **advanced_search_v3**
- **download_sample**

### Credentials
Credentials are loaded from a local file instead of being written here in plain text.
To learn how to creat the credentials file, see the **Storing and using the credentials** section in the [README file](./README.md)

In [ ]:
import json
import os
from ReversingLabs.SDK.a1000 import A1000


CREDENTIALS = json.load(open('credentials.json'))
HOST = CREDENTIALS.get("a1000").get("a1000_url")
TOKEN = CREDENTIALS.get("a1000").get("token")

cwd = os.getcwd()
upper_level = os.path.dirname(cwd)
USER_AGENT = json.load(open(os.path.join(upper_level, "user_agent.json")))["user_agent"]

a1000 = A1000(
    host=HOST,
    token=TOKEN,
    user_agent=USER_AGENT,
    verify=False
)

### 1. Batch-download function
This function will be used for downloading each batch of matched samples. Execute it so you can use it further on in the notebook.

In [ ]:
def download_batch(sample_list, download_path):
    for sample in sample_list:
        response = a1000.download_sample(sample_hash=sample.get("sha1"))
        
        with open(os.path.join(download_path, sample.get("sha1")), "wb") as file_handle:
            file_handle.write(response.content)

### 2. Main function
To retrieve batches of samples matched by the Advanced Search API and download them as files, use the `download_advanced_search_matches` function.  
This function accepts the following parameters:
- `query_string`: advanced search query
- `download_path`: needs to be a full path to the existing folder
- `ticloud`: use cloud results in the search query
- `start_search_date`: start date for the search (optional)
- `end_search_date`: end date for the search (optional)
- `sorting_criteria`: field used to sort the results
- `sorting_order`: 'desc' or 'asc'
- `records_per_page`: matches are retrieved in pages. this number defines how big each page will be
- `max_results`: the maximum number of matches you want retrieved and downloaded

Execute the function so you can use it multiple times.

In [ ]:
def download_advanced_search_matches(query_string, download_path, ticloud=False, start_search_date=None, 
                                     end_search_date=None, sorting_criteria=None, sorting_order="desc", 
                                     records_per_page=100, max_results=500):
    page_number = 1
    more_pages = True
    result_count = 0
    
    while more_pages:
        response = a1000.advanced_search_v3(
            query_string=query_string,
            ticloud=ticloud,
            start_search_date=start_search_date,
            end_search_date=end_search_date,
            page_number=page_number,
            records_per_page=records_per_page,
            sorting_criteria=sorting_criteria,
            sorting_order=sorting_order
        )
        
        resp_json = response.json()
        results = resp_json.get("rl", {}).get("web_search_api", {}).get("entries", [])
        more_pages = resp_json.get("rl", {}).get("web_search_api", {}).get("more_pages")
        
        result_count += len(results)
        page_number += 1
        
        if max_results:
            
            if result_count >= max_results:
                excesss = result_count - max_results
                cutoff = len(results) - excesss
                
                results = results[:cutoff]
                download_batch(sample_list=results, download_path=download_path)
                
                break
                
        download_batch(sample_list=results, download_path=download_path)
        
        if not more_pages:
            break

### 3. Run

In [ ]:
download_advanced_search_matches(
    query_string="change_me",
    download_path="/change/me",
    ticloud=False,
    records_per_page=20,
    max_results=100
)